In [2]:
import numpy as np
import awkward as ak
import uproot
import src.functions as fn
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import cm
import networkx as nx

In [3]:
filename = "/eos/user/b/bmaier/ticl_ml_hackathon/samples/landingzone/single_photon_pu0/ntuples_13848174_247.root"
file = uproot.open(filename)
tracksters = file["ticlNtuplizer/tracksters"]

In [4]:
tracksters.keys()

['event',
 'NClusters',
 'NTracksters',
 'time',
 'timeError',
 'regressed_energy',
 'raw_energy',
 'raw_em_energy',
 'raw_pt',
 'raw_em_pt',
 'barycenter_x',
 'barycenter_y',
 'barycenter_z',
 'trackster_barycenter_eta',
 'trackster_barycenter_phi',
 'EV1',
 'EV2',
 'EV3',
 'eVector0_x',
 'eVector0_y',
 'eVector0_z',
 'sigmaPCA1',
 'sigmaPCA2',
 'sigmaPCA3',
 'id_probabilities',
 'vertices_indexes',
 'vertices_x',
 'vertices_y',
 'vertices_z',
 'vertices_energy',
 'vertices_correctedEnergy',
 'vertices_correctedEnergyUncertainty',
 'vertices_multiplicity',
 'layer_cluster_seed']

In [77]:
file["ticlNtuplizer"].keys()

['tracksters;1',
 'clusters;1',
 'graph;1',
 'candidates;1',
 'trackstersMerged;1',
 'associations;1',
 'simtrackstersSC;1',
 'simtrackstersCP;1',
 'tracks;1']

In [5]:
vertices_indexes = tracksters['vertices_indexes'].array()
vertices_x = tracksters['vertices_x'].array()
vertices_y = tracksters['vertices_y'].array()
vertices_z = tracksters['vertices_z'].array()
vertices_E = tracksters['vertices_energy'].array()

In [41]:
len(ak.argmax(vertices_E,axis=1))
print(ak.argmax(vertices_E,axis=1)[0])
print(vertices_indexes[:,[0,1]])
print(vertices_indexes[:,0])
print(vertices_indexes[:,1])
print(vertices_E[13])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 3, ... 3, 3, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1]
[[[0, 2, 3, 6, 8, 14, 21, 23, 33, ... 463, 466, 467, 468, 474, 480, 482, 484, 495]]]
[[0, 2, 3, 6, 8, 14, 21, 23, 33, ... 178, 180, 191, 205, 215, 225, 229, 235, 248]]
[[1, 4, 10, 17, 24, 37, 45, 65, 85, ... 463, 466, 467, 468, 474, 480, 482, 484, 495]]
[[0.0389, 0.0312, 0.0318, 0.535, 0.463, 0.153, ... 1.08, 0.219], [0.296, 0.692]]


In [59]:
print(vertices_indexes[:,ak.argmax(vertices_E,axis=1)[0],:])

ValueError: in ListOffsetArray64 attempting to get 3, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/cpu-kernels/awkward_ListArray_getitem_next_array.cpp#L32)

In [42]:
n_tracksters = tracksters['NTracksters'].array()


In [73]:
print(n_tracksters)

[6, 3, 9, 6, 9, 6, 5, 2, 8, 4, 5, 15, 5, ... 4, 4, 2, 12, 6, 10, 7, 3, 3, 7, 10, 2]


In [44]:
print(vertices_E[13])

[[0.0389, 0.0312, 0.0318, 0.535, 0.463, 0.153, ... 1.08, 0.219], [0.296, 0.692]]


In [71]:
print(ak.count(vertices_E,axis=2))
print(ak.count(vertices_indexes[0],axis=0))
print(len(vertices_indexes))

[[62, 74, 15, 67, 48, 4], [35, 6, 33], ... 20, 5, 18, 52, 26, 20, 8, 22], [35, 37]]
[6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ... 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]
100


In [109]:

#n_events=len(n_tracksters)
n_events=10
for i in range(0,n_events):
    for j in range(n_tracksters[i]):
        offset=0
        if(vertices_z[i,j,0]<0):
            offset=320
            zmax=ak.min(vertices_z[i,j])
        else:
            offset=-320
            zmax=ak.max(vertices_z[i,j])
        argmax_E=ak.argmax(vertices_E[i,j])
        E_tot=ak.sum(vertices_E[i,j])
        print("Z position of highest energy: {}".format(vertices_z[i,j,argmax_E]+offset))
        print("Total energy: {:.2f}".format(E_tot))
        print("Max Z position: {}".format(zmax))
        ld=sum(vertices_z[i,j]*vertices_E[i,j])/offset*-1
        print("ld= {}".format(ld))
        print("sd={}".format(ld/E_tot))
        vEmax_x=vertices_x[i,j,argmax_E]
        vEmax_y=vertices_y[i,j,argmax_E]
        RvEmax=np.sqrt(vEmax_x**2+vEmax_y**2)
        R=np.sqrt(vertices_x[i,j]**2+vertices_y[i,j]**2)
        delta_R=sum((R-RvEmax)**2)
        print("delta_R={}".format(np.sqrt(delta_R)))
        delta_R_Eweighted=sum((R-RvEmax)**2*vertices_E[i,j]/vertices_E[i,j,argmax_E])
        print("delta_R_Eweighted={}".format(np.sqrt(delta_R_Eweighted)))
        
        
        

Z position of highest energy: -16.9527587890625
Total energy: 16.23
Max Z position: -358.687255859375
ld= 17.10402873983596
sd=1.053632328851408
delta_R=21.60005677655526
delta_R_Eweighted=5.716804880528014
Z position of highest energy: -13.982757568359375
Total energy: 388.47
Max Z position: -373.14898681640625
ld= 409.52237654393156
sd=1.0541917992345993
delta_R=26.025770286707967
delta_R_Eweighted=5.339333339653046
Z position of highest energy: -19.922760009765625
Total energy: 2.14
Max Z position: -352.74725341796875
ld= 2.2686899104965743
sd=1.0603448507916562
delta_R=8.135115524710336
delta_R_Eweighted=3.1716598666323135
Z position of highest energy: 16.9527587890625
Total energy: 402.81
Max Z position: 361.6572570800781
ld= 425.74127207414784
sd=1.0569286848585244
delta_R=32.10275909953858
delta_R_Eweighted=4.3864697441572345
Z position of highest energy: 29.777252197265625
Total energy: 5.54
Max Z position: 360.7127380371094
ld= 6.011761554693467
sd=1.0846753774653615
delta_R=2

Max Z position: -358.687255859375
ld= 70.6638768721447
sd=1.0480196809724396
delta_R=16.318107584545817
delta_R_Eweighted=3.0769423043704287
Z position of highest energy: 13.982757568359375
Total energy: 65.51
Max Z position: 357.74273681640625
ld= 68.41826389673454
sd=1.0444247377190077
delta_R=22.657517031291647
delta_R_Eweighted=3.7416452826297864
Z position of highest energy: -11.01275634765625
Total energy: 302.19
Max Z position: -361.6572570800781
ld= 316.6628897889419
sd=1.0478862034913878
delta_R=23.71852345227123
delta_R_Eweighted=3.8657070816068413
Z position of highest energy: -13.982757568359375
Total energy: 0.86
Max Z position: -337.8972473144531
ld= 0.8928439231786645
sd=1.0431665499906264
delta_R=1.3745178632655304
delta_R_Eweighted=0.5520417387919118
Z position of highest energy: -19.922760009765625
Total energy: 2.00
Max Z position: -343.8372497558594
ld= 2.112982147997247
sd=1.0569301165148146
delta_R=5.188828121086572
delta_R_Eweighted=2.1999966045602313
Z position 

[-322, -323, -323, -325, -325, -326, -328, ... -353, -353, -353, -356, -356, -359]
[0.0461, 0.0623, 0.142, 0.868]
ld= 3.0393529461197786
[-322, -323, -325, -326, -328, -329, -331, ... -359, -361, -362, -362, -368, -373]
[0.0461, 0.0623, 0.142, 0.868]
ld= 4.989654142496958
[-332, -334, -335, -337, -338, -338, -340, ... -343, -344, -346, -346, -349, -353]
[0.0461, 0.0623, 0.142, 0.868]
ld= 2.275748593262705
[322, 323, 323, 325, 326, 326, 328, 328, ... 355, 356, 356, 358, 359, 361, 362, 362]
[0.0461, 0.0623, 0.142, 0.868]
ld= 62.05742817320438
[332, 337, 337, 338, 338, 338, 338, 340, ... 355, 355, 356, 358, 358, 359, 361, 361]
[0.0461, 0.0623, 0.142, 0.868]


ValueError: in ListOffsetArray64 attempting to get 4, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/cpu-kernels/awkward_NumpyArray_getitem_next_at.cpp#L21)

In [3]:
def ld(vertices_z,vertices_E):
    if(vertices_z[0]<0):
        offset=320
    else:
        offset=-320
    
    ldVal=sum((vertices_z+offset)*vertices_E)
    return ldVal

def delta_R(vertices_x,vertices_y,vertices_E,Eweighted=False):
    argmax_E=ak.argmax(vertices_E)
                       
    vEmax_x=vertices_x[argmax_E]
    vEmax_y=vertices_y[argmax_E]
    RvEmax=np.sqrt(vEmax_x**2+vEmax_y**2)
    
    R=np.sqrt(vertices_x**2+vertices_y**2)
    
    if(Eweighted):
        delta_R=sum((R-RvEmax)**2*vertices_E/vertices_E[argmax_E])
    else:
        delta_R=sum((R-RvEmax)**2)
        
    return delta_R
                       
def maxE_z(vertices_z,vertices_E):
    argmax_E=ak.argmax(vertices_E)
    if(vertices_z[0]<0):
        maxE_z=-1*(vertices_z[argmax_E]+320)
    else:
        maxE_z=(vertices_z[argmax_E]-320)
    return maxE_z
    
def sd(vertices_z,vertices_E):
    ldVal=ld(vertices_z,vertices_E)
    E_tot=ak.sum(vertices_E)
    return ldVal/E_tot

def maxAbsZ(vertices_z):
    if(vertices_z[0]<0):
        maxZ=ak.min(vertices_z)
    else:
        maxZ=ak.max(vertices_z)
    return abs(maxZ)


In [4]:
filename2 = "/afs/cern.ch/user/d/dmagdali/CernCentralityProject/tracksters_ds_10e.root"
file2 = uproot.open(filename2)
tracksters10e=file2['tracksters']

In [ ]:
print(tracksters10e.keys())
vertices_E=tracksters10e['vertices_energy'].array()
vertices_Labels=tracksters10e['trackster_label'].array()
vertices10e_indexes=tracksters10e['vertices_indexes'].array()
vertices10e_x = tracksters10e['vertices_x'].array()
vertices10e_y = tracksters10e['vertices_y'].array()
vertices10e_z = tracksters10e['vertices_z'].array()
vertices10e_E = tracksters10e['vertices_energy'].array()

edges10e = fn.edgeBuilderNNTr(vertices10e_indexes, vertices10e_x, vertices10e_y, vertices10e_z, vertices10e_E)

['nvertices_x', 'vertices_x', 'nvertices_y', 'vertices_y', 'nvertices_z', 'vertices_z', 'nvertices_energy', 'vertices_energy', 'nvertices_indexes', 'vertices_indexes', 'trackster_label', 'trackster_event']


In [ ]:
tooSmall = ak.num(vertices10e_indexes,axis=-1) > 2

In [ ]:
vertices10e_indexes_g = vertices10e_indexes[tooSmall]
vertices10e_x_g = vertices10e_x[tooSmall]
vertices10e_y_g = vertices10e_y[tooSmall]
vertices10e_z_g = vertices10e_z[tooSmall]
vertices10e_E_g = vertices10e_E[tooSmall]

In [161]:
#n_events=len(n_tracksters)

for j in range(len(vertices10e_indexes_g)):
    if(vertices10e_z_g[j,0]<0):
        offset=320
    else:
        offset=-320
    print(vertices10e_z_g[j]+offset)
    print(vertices10e_E_g[j])
    ld=sum((vertices10e_z_g[j]+offset)*vertices10e_E_g[j])
    print("ld= {}".format(ld))
    print("E tot = {}".format(sum(vertices10e_E_g[j])))

[37.7, 38.7, 40.7, 41.7]
[0.0461, 0.0623, 0.142, 0.868]
ld= 46.10154204706578
E tot = 1.118688140064478
[29.8, 31.8, 31.8, 32.7, 34.8, 35.7, 37.7, 37.7, 38.7, 40.7]
[0.0429, 0.0433, 0.0756, 0.159, 1.11, 0.113, 0.0344, 0.47, 0.0748, 0.0708]
ld= 77.64077365642095
E tot = 2.1916893497109413
[11, 11, 14, 17, 20.9, 20.9, 22.9, ... 25.9, 28.8, 28.8, 28.8, 29.8, 29.8, 31.8]
[0.0855, 0.033, 0.11, 0.668, 0.0815, 0.317, ... 0.0465, 0.157, 0.0384, 0.031, 0.0323]
ld= 45.43309566103051
E tot = 2.162506729364395
[17, 17.9, 17.9, 19.9, 20.9, 22.9, 23.8, 23.8, 26.8, 29.8]
[0.0926, 0.0896, 0.0194, 0.0613, 0.198, 0.0244, 0.754, 0.111, 0.16, 0.0647]
ld= 36.27800882557733
E tot = 1.5754130128771067
[19.9, 20.9, 23.8, 23.8, 28.8, 31.8]
[0.125, 0.0295, 0.542, 0.0283, 0.259, 0.0817]
ld= 26.762808622339662
E tot = 1.0654846858233213
[31.8, 32.7, 35.7, 38.7, 40.7]
[0.0558, 0.271, 0.669, 0.0764, 0.0679]
ld= 40.25108347934929
E tot = 1.1397269070148468
[23.8, 26.8, 28.8, 29.8, 29.8, 31.8, 31.8, 32.7, 34.8, 35.7,

E tot = 107.11403634771705
[3.05, 5.07, 6.02, 6.02, 8.04, 8.99, 8.99, ... 35.7, 37.7, 38.7, 38.7, 40.7, 41.7]
[0.228, 1.04, 2.32, 0.0736, 3.76, 5.69, ... 2.08, 2.31, 1.19, 0.479, 0.819, 1.54]
ld= 3996.7743004601753
E tot = 187.72840621322393
[5.07, 6.02, 8.04, 8.04, 8.99, 8.99, 11, ... 38.7, 40.7, 41.7, 41.7, 47.7, 53.1]
[0.664, 1.59, 0.0381, 3.29, 0.196, 5.49, ... 2.84, 2.26, 0.344, 5.51, 1.16, 0.113]
ld= 7311.379973759787
E tot = 307.6727590560913
[3.05, 5.07, 6.02, 8.04, 8.99, 11, 12, ... 31.8, 32.7, 32.7, 34.8, 37.7, 37.7, 40.7]
[0.712, 1.2, 2.66, 5.62, 6.06, 8.82, ... 0.12, 0.0574, 0.0332, 0.131, 0.109, 0.0542]
ld= 1133.3194800899337
E tot = 77.87347582355142
[5.07, 6.02, 8.04, 8.04, 8.99, 11, 11, ... 32.7, 34.8, 34.8, 35.7, 35.7, 38.7, 41.7]
[0.823, 3.05, 5.68, 0.167, 10.6, 0.0764, ... 0.535, 0.151, 0.105, 0.07, 0.243, 0.335]
ld= 2703.553508253213
E tot = 164.7347283437848
[3.05, 5.07, 6.02, 8.04, 8.99, 11, 12, ... 32.7, 34.8, 34.8, 35.7, 37.7, 38.7, 40.7]
[0.366, 2.16, 3.97, 4.7

E tot = 0.9797255732119083
[14.9, 19.9, 22.9, 23.8, 25.9, 26.8, 28.8, 29.8, 31.8]
[0.0855, 0.135, 0.162, 0.143, 0.268, 0.107, 0.606, 0.068, 0.18]
ld= 46.11979470541064
E tot = 1.7548908740282059
[38.7, 41.7, 47.7]
[0.0579, 0.48, 0.96]
ld= 68.00100454627955
E tot = 1.4973330087959766
[29.8, 29.8, 29.8, 29.8, 31.8, 31.8, 31.8, ... 34.8, 34.8, 35.7, 35.7, 37.7, 38.7]
[0.146, 0.11, 0.0391, 0.0519, 0.0646, ... 0.0855, 0.0976, 0.179, 0.839, 0.333]
ld= 90.1588494297178
E tot = 2.556980052962899
[11, 14.9, 14.9, 14.9, 17.9, 17.9, 19.9, ... 25.9, 26.8, 28.8, 29.8, 31.8, 31.8]
[0.0498, 0.132, 0.0572, 0.0363, 0.102, ... 0.0418, 0.0532, 0.0249, 0.0424, 0.0667]
ld= 62.00692852606062
E tot = 2.8026513382792473
[19.9, 20.9, 22.9, 23.8, 28.8, 31.8]
[0.0493, 0.647, 0.193, 0.0894, 0.0736, 0.0291]
ld= 24.078633359395212
E tot = 1.0812834613025188
[17, 20.9, 20.9, 23.8, 25.9, 25.9, 26.8, 28.8, 28.8, 29.8, 31.8, 32.7, 37.7]
[0.0647, 0.057, 0.0704, 0.116, 0.04, 0.034, ... 0.0595, 0.0449, 2.88, 0.129, 0.0825

E tot = 147.90487848222256
[5.07, 6.02, 6.02, 8.04, 8.99, 11, 12, ... 38.7, 40.7, 40.7, 40.7, 40.7, 41.7, 41.7]
[0.332, 0.0445, 1.4, 3.81, 8.56, 11.9, ... 0.128, 0.147, 0.0935, 0.0817, 0.367]
ld= 3192.0992348540567
E tot = 175.13825225830078
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 35.7, 37.7, 37.7, 38.7, 38.7, 38.7, 40.7]
[0.47, 2.48, 2.73, 5.52, 10, 12.9, ... 0.135, 0.333, 0.109, 0.353, 0.0964, 0.031]
ld= 2639.0760141046
E tot = 149.28064727038145
[17, 17.9, 19.9, 20.9, 20.9, 22.9, 23.8, ... 40.7, 41.7, 41.7, 41.7, 47.7, 53.1]
[1.77, 4.53, 6.56, 5.31, 0.0801, 9.47, ... 0.031, 0.551, 0.117, 0.874, 0.265, 0.398]
ld= 2854.934395581374
E tot = 104.44499787501991
[5.07, 6.02, 8.04, 8.99, 8.99, 11, 11, ... 37.7, 37.7, 37.7, 38.7, 38.7, 40.7, 40.7]
[0.698, 1.8, 4.04, 6.35, 0.0835, 8.54, ... 0.34, 0.0458, 0.154, 0.062, 0.097, 0.0357]
ld= 2230.745429950406
E tot = 126.50598984025419
[5.07, 6.02, 6.02, 8.04, 8.99, 8.99, 11, ... 35.7, 35.7, 37.7, 38.7, 40.7, 41.7]
[0.236, 1.6, 0.0626, 3.78, 5.6

E tot = 39.28819878026843
[5.07, 6.02, 8.04, 8.04, 8.99, 11, 12, ... 38.7, 38.7, 40.7, 40.7, 41.7, 41.7, 41.7]
[0.411, 2.29, 4.76, 0.0364, 8.75, 10.8, ... 0.0344, 0.256, 0.408, 0.291, 0.197]
ld= 3505.1821848098803
E tot = 184.45557595416903
[5.07, 6.02, 8.04, 8.99, 11, 11, 12, ... 29.8, 29.8, 31.8, 32.7, 32.7, 34.8, 35.7]
[0.0586, 0.419, 0.989, 1.85, 0.0538, 2.77, ... 0.173, 0.036, 0.127, 0.055, 0.0894]
ld= 523.9180526348347
E tot = 31.46845803782344
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 35.7, 35.7, 35.7, 37.7, 37.7, 38.7, 41.7]
[0.417, 0.929, 2.98, 4.02, 5.94, 9.61, ... 0.0558, 0.258, 0.0303, 0.0336, 0.184]
ld= 2431.9113569648653
E tot = 132.7118365690112
[2.1, 3.05, 5.07, 6.02, 8.04, 8.04, 8.99, ... 31.8, 31.8, 31.8, 32.7, 32.7, 32.7]
[0.0706, 0.321, 0.654, 2.09, 2.81, 0.137, ... 0.0431, 0.0727, 0.035, 0.0923, 0.0458]
ld= 967.2288141534475
E tot = 63.62593914195895
[5.07, 6.02, 8.04, 8.04, 8.99, 8.99, 8.99, ... 37.7, 37.7, 37.7, 38.7, 40.7, 40.7]
[0.111, 0.28, 1.28, 0.176, 0.0444,

...

E tot = 34.28191373869777
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 37.7, 37.7, 38.7, 40.7, 40.7, 40.7, 41.7]
[0.153, 0.685, 2.17, 3.61, 6.81, 7.61, ... 0.32, 0.0696, 0.211, 0.0732, 0.106, 0.336]
ld= 1838.122545298065
E tot = 98.79940000921488
[5.07, 6.02, 8.04, 8.99, 11, 11, 12, ... 34.8, 35.7, 35.7, 35.7, 37.7, 38.7, 41.7]
[0.315, 1.1, 2.92, 5.01, 0.0311, 8.65, ... 0.32, 0.163, 0.51, 0.167, 0.0307, 0.378]
ld= 2820.7527488619903
E tot = 155.1832487117499
[6.02, 8.04, 8.99, 11, 12, 14, 14.9, ... 32.7, 34.8, 35.7, 37.7, 40.7, 41.7, 47.7]
[0.0417, 0.773, 2.36, 5.02, 5.19, 5.96, ... 0.169, 0.0615, 0.159, 0.243, 0.129]
ld= 1220.3907046396066
E tot = 64.84960597008467
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 28.8, 28.8, 31.8, 32.7, 32.7, 32.7, 35.7]
[0.241, 0.634, 1.08, 2.22, 3.37, 3.2, ... 1.07, 0.497, 0.185, 0.204, 0.224, 0.121]
ld= 496.4277322961131
E tot = 29.245220189914107
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 32.7, 34.8, 34.8, 35.7, 35.7, 37.7, 40.7]
[0.456, 1.18, 2.39, 4.39, 6.85,

[0.0489, 0.0263, 0.213, 0.344, 0.101, 0.0692, 0.212, 1.46, 0.563]
ld= 136.0083977228819
E tot = 3.0324425660073757
[26.8, 31.8, 31.8, 32.7, 35.7, 38.7]
[0.123, 0.172, 0.655, 0.61, 0.0761, 0.095]
ld= 55.94565794974301
E tot = 1.7303555980324745
[8.99, 11, 11, 12, 12, 14.9, 17, 17, 19.9, 20.9, 22.9, 22.9, 25.9]
[0.0397, 0.0296, 0.0761, 0.0734, 0.0667, ... 0.112, 0.185, 0.362, 0.033, 0.029]
ld= 36.630282220320396
E tot = 2.049716491252184
[12, 14, 14, 14, 14.9, 17, 17.9, 17.9, ... 22.9, 22.9, 23.8, 23.8, 23.8, 26.8, 28.8]
[0.0606, 0.0727, 0.134, 0.347, 0.0613, ... 0.0438, 0.267, 0.0761, 0.0424, 0.0478]
ld= 53.62461937847115
E tot = 2.7999164536595345
[8.99, 12, 17, 17, 17.9, 17.9, 17.9, ... 26.8, 26.8, 26.8, 28.8, 28.8, 29.8, 32.7]
[0.0613, 0.0566, 0.102, 0.193, 0.373, 0.207, ... 0.0384, 0.121, 0.154, 0.0572, 0.033]
ld= 71.55265104483681
E tot = 3.2261944375932217
[17, 17, 17, 17.9, 19.9, 20.9, 22.9, ... 28.8, 29.8, 29.8, 31.8, 31.8, 31.8, 34.8]
[0.0754, 0.0343, 0.343, 0.0458, 0.109, ... 

[2.1, 5.07, 6.02, 6.02, 8.04, 8.99, 11, ... 35.7, 35.7, 37.7, 38.7, 40.7, 40.7, 41.7]
[0.0361, 0.275, 0.0896, 1.69, 4.93, 7.18, ... 0.275, 0.0976, 0.253, 0.0438, 0.147]
ld= 2414.427361492888
E tot = 144.37936600856483
[3.05, 5.07, 6.02, 6.02, 8.04, 8.04, 8.99, ... 34.8, 35.7, 37.7, 38.7, 40.7, 41.7]
[0.292, 1.4, 2.56, 0.0946, 6.39, 0.0805, ... 0.139, 0.157, 0.189, 0.19, 0.144, 0.145]
ld= 1613.2602413414975
E tot = 104.1068855188787
[5.07, 6.02, 8.04, 8.99, 8.99, 11, 12, ... 37.7, 38.7, 38.7, 40.7, 40.7, 41.7, 41.7]
[0.285, 0.805, 1.53, 0.0875, 3.57, 4.75, ... 0.0275, 0.142, 0.282, 0.101, 0.25, 0.27]
ld= 1991.401110009848
E tot = 102.30146226286888
[5.07, 6.02, 8.04, 8.04, 8.99, 11, 12, ... 35.7, 35.7, 37.7, 37.7, 38.7, 38.7, 40.7]
[0.095, 0.159, 0.0923, 1.08, 2.82, 4.94, ... 0.0545, 0.106, 0.0808, 0.137, 0.0303]
ld= 1432.9487994235196
E tot = 78.35807150229812
[5.07, 6.02, 8.04, 8.04, 8.99, 8.99, 11, ... 35.7, 35.7, 35.7, 37.7, 40.7, 47.7]
[0.378, 1.26, 0.126, 3.99, 8.01, 0.176, ... 0.

[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 34.8, 35.7, 35.7, 35.7, 35.7, 37.7, 37.7]
[0.29, 1.8, 2.91, 7.05, 10.4, 12.5, ... 0.145, 0.0418, 0.0343, 0.0734, 0.0714]
ld= 1984.7891468009207
E tot = 119.86127548478544
[6.02, 8.04, 8.99, 11, 11, 11, 12, 12, ... 37.7, 37.7, 37.7, 37.7, 37.7, 38.7, 41.7]
[0.606, 1.73, 3.25, 0.0976, 5.34, 0.237, ... 0.0485, 0.0498, 0.0613, 0.206, 0.259]
ld= 1629.807995109156
E tot = 87.48722929880023
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 32.7, 32.7, 32.7, 32.7, 34.8, 34.8, 35.7]
[0.518, 2.27, 4.28, 6.42, 7.59, 11.1, ... 0.235, 0.163, 0.158, 0.0519, 0.159, 0.187]
ld= 1827.663795153096
E tot = 109.76951764523983
[5.07, 6.02, 8.04, 8.99, 11, 11, 12, ... 29.8, 31.8, 32.7, 34.8, 34.8, 35.7, 38.7]
[0.837, 1.96, 3.4, 4.97, 6.01, 0.0545, ... 0.0424, 0.139, 0.0909, 0.035, 0.163]
ld= 1007.8762836593158
E tot = 65.14089808613062
[6.02, 8.04, 8.99, 11, 12, 14, 14.9, ... 35.7, 35.7, 35.7, 37.7, 37.7, 38.7, 38.7]
[0.282, 1.83, 3.73, 7.7, 6.37, 8.95, ... 0.218, 0.0431, 0.05

[0.189, 0.0788, 0.531, 1.53, 3.99, 5.37, ... 0.0916, 0.0492, 0.197, 0.115, 0.501]
ld= 2580.8164774554903
E tot = 129.85331421159208
[3.05, 5.07, 6.02, 8.04, 8.04, 8.04, 8.99, ... 34.8, 34.8, 34.8, 34.8, 35.7, 37.7]
[0.284, 1.27, 2.86, 0.0384, 4.83, 0.0384, ... 0.116, 0.317, 0.0613, 0.0478, 0.164]
ld= 1406.934021087732
E tot = 88.38824052736163
[5.07, 6.02, 6.02, 8.04, 8.99, 8.99, 11, ... 35.7, 35.7, 35.7, 37.7, 37.7, 38.7]
[0.803, 2.63, 0.0332, 5.52, 0.0848, 8.4, ... 0.448, 0.0267, 0.322, 0.146, 0.171]
ld= 2736.286558727943
E tot = 159.93784037791193
[5.07, 6.02, 8.04, 8.99, 11, 12, 14, ... 32.7, 32.7, 34.8, 34.8, 34.8, 35.7, 37.7]
[0.723, 1.4, 3.46, 5.2, 7.61, 7.71, ... 0.154, 0.0481, 0.393, 0.0263, 0.102, 0.0283]
ld= 972.0899745620646
E tot = 64.59590425342321
[3.05, 5.07, 6.02, 8.04, 8.99, 8.99, 11, ... 32.7, 34.8, 35.7, 37.7, 37.7, 38.7]
[0.11, 0.738, 2.01, 4.6, 0.0849, 7.38, ... 0.0574, 0.0574, 0.195, 0.0445, 0.0332]
ld= 1687.98989947819
E tot = 109.79294404014945
[2.1, 3.05, 5.07

In [ ]:
comLd=[]
inComLd=[]
comSd=[]
inComSd=[]
comDelta_R=[]
inComDelta_R=[]
comTotE=[]
inComTotE=[]
comDelta_R_Eweighted=[]
inComDelta_R_Eweighted=[]
comMaxE_z=[]
inComMaxE_z=[]
comMaxAbs_z=[]
inComMaxAbs_z=[]

#N_tr = len(vertices10e_indexes_g)
N_tr=1000

for i in range(N_tr):
    v_ind=vertices10e_indexes_g[i]
    v_x=vertices10e_x_g[i] 
    v_y=vertices10e_y_g[i]
    v_z=vertices10e_z_g[i]
    v_E=vertices10e_E_g[i] 
    """
    edges_1 = fn.edgeBuilderNew(
        v_ind,
        v_x, 
        v_y, 
        v_z, 
        v_E
    )
    edges_1 = ak.flatten(edges_1[ak.num(edges_1) > 0].to_list())
    """
    if(vertices_Labels[i]==1):
        comLd.append(ld(v_z,v_E))
        comSd.append(sd(v_z,v_E))
        comDelta_R.append(delta_R(v_x,v_y,v_E))
        comDelta_R_Eweighted.append(delta_R(v_x,v_y,v_E,True))
        comTotE.append(ak.sum(v_E))
        comMaxE_z.append(maxE_z(v_z,v_E))
        comMaxAbs_z.append(maxAbsZ(v_z))
    else:
        inComLd.append(ld(v_z,v_E))
        inComSd.append(sd(v_z,v_E))
        inComDelta_R.append(delta_R(v_x,v_y,v_E))
        inComDelta_R_Eweighted.append(delta_R(v_x,v_y,v_E,True))
        inComTotE.append(ak.sum(v_E))
        inComMaxE_z.append(maxE_z(v_z,v_E))
        inComMaxAbs_z.append(maxAbsZ(v_z))



In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comLd, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComLd, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("ld",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comSd, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComSd, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("sd",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comDelta_R, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComDelta_R, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("Delta_R",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comDelta_R_Eweighted, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComDelta_R_Eweighted, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("Delta_R_Eweighted",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comTotE, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComTotE, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("Total energy",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comMaxE_z, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComMaxE_z, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("EMax Z position",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(comMaxAbs_z, bins=50, color='b', alpha=0.7,label='Complete')
plt.hist(inComMaxAbs_z, bins=50, color='r', alpha=0.7,label='Incomplete')
plt.legend()
plt.ylabel("N",fontsize=20)
plt.xlabel("Maximum Z position",fontsize=20)
plt.title("10e dataset",fontsize=20)
plt.show()
